# Modified Dataset

The purpose of this notebook is to get an overview on the expanded Restaurants data. Especially the parameters extracted from the Google API are of interest. An elementary understanding of the parameters is key to comprehend the more specific analyzes later on.

## Preparing the Dataframe

A few refactoring steps will make the data more clear. We start by loading the merged dataframe from previous notebook:

In [ ]:
import pandas as pd

data = pd.read_pickle(r'data/detailed_restaurants.pkl')
df = pd.DataFrame(data)

Some columns are not of interest in our analyzes and can be dropped. Additionally, we set a new index, change column names and reorder just these:

In [ ]:
# Drop columns not of interest
df.drop("art", axis=1, inplace=True)  # all equal "Gastronomie"
df.drop("fon", axis=1, inplace=True)
df.drop("place_id", axis=1, inplace=True)


# Rename columns
df.rename(columns={'unique_id': "id",  # rename
                   'name': "name",
                   'strasse_nr': "strasse_nr",
                   'bezirk': "bezirk",
                   'lat': "lat",
                   'lng': "lng",
                   'lieferung': "delivery",
                   'rating': "rating",
                   'user_ratings_total': "nr_ratings",
                   'price_level': "price_lvl",
                   'types': "types"},
          inplace=True)

# Reorder columns
df = df[['id', 'name', 'strasse_nr', 'bezirk', 'lat', 'lng', 'types', 'delivery', 'price_lvl', 'nr_ratings', 'rating']]

# Remove duplicate information
df['bezirk'] = df['bezirk'].str.replace("Bezirk ", "")

# Set new index according to original csv file
df.set_index('id', inplace=True)

display(df)

<br>Unfortunately, some restaurants are missing entries. While that is fine for optional parameters like 'price_lvl', including places without any rating will bias the results. It is unlikely for a running restaurant business to have no Google rating. Therefore, one can assume, that either the API request wasn't succesful, the wrong output was matched or that the place is not a restaurant. For latter we can control by evaluating the 'types' parameter:

In [ ]:
# Create empty list
removed_restaurants = []

# Get index of restaurants with zero reviews
idx_list = df[df.nr_ratings == 0].index
idx_list2 = df[df.nr_ratings == None].index

# Append removed restaurants to list
for idx in idx_list:
    removed_restaurants.append(df.loc[idx, 'name'])
for idx in idx_list2:
    removed_restaurants.append(df.loc[idx, 'name'])

# Remove restaurants with zero ratings
df.drop(df[df.nr_ratings == 0].index, inplace=True)
df.drop(df[df.nr_ratings == None].index, inplace=True)
 
# Remove and append restaurants with wrong type
for idx, restaurant in df.iterrows():
    types = restaurant['types']
    try:
        if any(type in "restaurant and food and cafe and bakery and bar and meal_takeaway" for type in types):  # substring search
            pass
        else:
            df.drop(idx, axis=0, inplace=True)
            removed_restaurants.append(restaurant['name'])
    except TypeError:  # NaN = integer and needs to be dropped
        df.drop(idx, axis=0, inplace=True)
        removed_restaurants.append(restaurant['name'])

# Remove duplicates
list(set(removed_restaurants))
           
print("Number of restaurants removed: " + str(len(removed_restaurants)))
print("\n")
print(removed_restaurants)

<br>As predicted some of the removed places are no restaurants (e.g. Foodlocker). There might have been a fault in the admission of the place to the dataset or other categories did not fit either. Delivery service platforms (e.g. abholservice24.de) and catering companies (e.g. Hefter Partyservice) are not supposed to exist in the "Gastronomie" category.

In addition, some places that are actual restaurants were removed. Most of them missed the Google rating within the API result. When googling "Pizza Hof" you'll see that there are 197 ratings with an average of 4.5 stars for that place. Unfortunately, no address is saved. Due to that and other reasons (i.e. moved/closed business, new opening), some restaurants with valid ratings had to be omitted from the analysis.

At last, the datatype of the columns is changed according to their values. For price level we take over Googles interpretation:

In [ ]:
# Change type of columns according to entries
df[['lat', 'lng', 'rating']] = df[['lat', 'lng', 'rating']].apply(pd.to_numeric)  # float
df[['nr_ratings']] = df[['nr_ratings']].astype(int)  # integer

# Refactor price level values
df.loc[df['price_lvl'] == 0, 'price_lvl'] = "Free"  # there should not be any entry
df.loc[df['price_lvl'] == 1, 'price_lvl'] = "Inexpensive"
df.loc[df['price_lvl'] == 2, 'price_lvl'] = "Moderate"
df.loc[df['price_lvl'] == 3, 'price_lvl'] = "Expensive"
df.loc[df['price_lvl'] == 4, 'price_lvl'] = "Very Expensive"
df['price_lvl'] = df['price_lvl'].fillna("Unknown")  # removes NaN

# Refactor 'lieferservice' to boolean
df.loc[df['delivery'] == "WAHR", 'delivery'] = 1
df.loc[df['delivery'] == "FALSCH", 'delivery'] = 0
df['delivery'] = df['delivery'].astype('bool')

# Export modified dataframe for later use
df.to_pickle(r'data/modif_restaurants.pkl')
df.to_csv(r'data/modif_restaurants.csv', sep=';', encoding='utf-8', index=True)

display(df)

## Statistical Overview

Before moving to more advanced research questions, the general composition of the Restaurants dataset shall be examined. Of special interest are the paramaters received through the Google API (i.e. 'price_lvl', 'rating', 'nr_ratings').

Data Visualization requires the following packages:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression

### General

Since we removed incomplete rows and replaced missing values for 'price_lvl' with a string (i.e. "Unknown"), there should not be any NaN values left in our data. Pandas dataframes require columns to hold uniform data. Knowledge of the column type is important for later analyzes:

In [ ]:
# Create series with NaN count
nan_series = df.isna().sum()

# Create series with data type
types_series = df.dtypes

# Merge series to dataframe
parameter_overview_df = pd.concat([nan_series, types_series], axis=1)
parameter_overview_df.columns = ['NaN_count', 'dtype']
                                  
print(parameter_overview_df)
print("\n")
print("Shape of the dataframe: " + str(df.shape))

There is valid data for **674 restaurants** on **10 parameters** of interest.

### Parameters

Lets have a closer look on the distribution of each relevant numeric parameter:

**1) Delivery Service**

In [ ]:
print(df['delivery'].value_counts())

# Series with frequency of Price Level
delivery_distribution = df['delivery'].value_counts()

# Creating bar plot
X = delivery_distribution.index 
Y = delivery_distribution.values
plt.pie(Y, labels=X, autopct='%.1f')
plt.show()

**2) Price Level**

In [ ]:
%matplotlib notebook

print(df['price_lvl'].value_counts())

# Series with frequency of Price Level
price_lvl_distribution = df['price_lvl'].value_counts()
price_lvl_distribution = price_lvl_distribution.reindex(index=['Unknown',  # ascending order
                                                               'Inexpensive',
                                                               'Moderate',
                                                               'Expensive',
                                                               'Very Expensive'])

# Creating bar plot
X = price_lvl_distribution.index 
Y = price_lvl_distribution.values
plt.xlabel('price_lvl')
plt.ylabel('frequency')
plt.bar(X, Y, width=0.5)
plt.show()

Most restaurants are in the "Moderate" price range. There are more "Inexensive" places than "Expensive" and "Very Expensive" ones. Google itself claims that the exact price range indicated by 'price_lvl' will vary from region to region. Therefore, one cannot specify any boundaries (e.g. 25€ to 35€ for a main course is expensive). Missing entries (i.e. "Unknown") might be caused due to a lack of reviews.

**3) Rating**

In [ ]:
%matplotlib notebook

# Series with frequency of Ratings
rating_distribution = df['rating'].value_counts()
rating_distribution = rating_distribution.sort_index(ascending=True)  # sort by value ascending

# Statistical overview on parameter 'rating'
describe_df = df['rating'].describe()
rating_mean = describe_df.loc['mean']  # saving mean for later use

print(describe_df.round(decimals=2))
print("median: " + str(df['rating'].median()))

# Creating bar plot
X = rating_distribution.index
Y = rating_distribution.values
plt.xlabel("rating")
plt.ylabel("frequency")
plt.bar(X, Y, width=0.3)
plt.axvline(rating_mean, color='k', linestyle='dashed', linewidth=1)  # includes mean in plot
plt.show()

Even though 'rating' is a discrete parameter, plotting its frequencies suggests a bell shaped distribution. While there are only a few places with extraordinary good (i.e. 4.75 to 5.0 stars) and bad (i.e. below 4.0 stars) ratings, the majority of restaurants have ratings between 4.0 and 4.75 stars. We try to examine the nature and possible bias of just this distribution later on.

**4) Number of Ratings**

In [ ]:
%matplotlib notebook

print(df['nr_ratings'].describe().round(decimals=1))
print("median: " + str(df['nr_ratings'].median()))

# Creating bins for grouped frequencies
bins_tpl = [(1, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600), (600, 700), (700, 800), (800, 900), (900, 1000), (1000, 2000), (2000, 15000)]
bins = pd.IntervalIndex.from_tuples(bins_tpl)  # converts tuple to IntervalIndex, half-open by default
cat_object = pd.cut(df['nr_ratings'], bins)  # creates categorial object

# Statistical overview on bins
nr_ratings_distribution = pd.value_counts(cat_object)
nr_ratings_distribution = nr_ratings_distribution.sort_index(ascending=True)
nr_ratings_distribution.index = nr_ratings_distribution.index.map(str)  # converts bins to strings and therefore categorial object to series

# Creating bar plot
X = nr_ratings_distribution.index
y = nr_ratings_distribution.values
fig, ax = plt.subplots()
fig.autofmt_xdate()  # format x-axis label
plt.xlabel("nr_ratings")
plt.ylabel("frequency")
plt.bar(X, y, width=0.5)
plt.show()

Most places have been reviewed between 1 and 100 times. With ascending intervals, the frequency declines. Keep in mind that the last two intervals are defined on a larger range. Worth mentioning is the disparity between mean (447.3) and median (274).

### Correlation

Of interest in our analyzes are the factors affecting a restaurants rating. Exposing a true causal effect is the desire of every Data Scientist. This however, is only possible under lab circumstances in an ideal randomized controlled experiment. "Real world" data has to be interpreted with care.

Regarding the Restaurants data this means, that there might be a wide range of parameters that end up influencing the rating of a place. One cannot tell to what degree the price, food type or location end up affecting the rating, since there are a lot of other possible factors and biases. Therefore, one needs to settle for less. The aim is to find patterns in the data, indicating a correlation and a possible causal effect.

**1) Rating and Number of Ratings**

This correlation indicates, whether restaurants with a lot of reviews tend to have better or worse ratings: 

In [ ]:
%matplotlib notebook

# Simple linear regression of number of ratings on rating
X = df['nr_ratings'].to_numpy()  # independent variable
y = df['rating'].to_numpy()  # dependent variable
X = X.reshape(-1, 1)
y = y.reshape(-1, 1)
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, y)  # perform regression
y_pred = linear_regressor.predict(X)  # make predictions

# Create plot
plt.scatter(X, y)
plt.plot(X, y_pred, color='red')
plt.show()

Linear regression is a convenient tool for revealing correlations. When there is no data on some intervals, the regression line might be a good enough estimator but without any statement about parameter coherence. Therefore, the outlier with over 12000 ratings needs to be omitted. Additionally, the results shall be presented in an adequate way:

In [ ]:
%matplotlib notebook

# Remove outlier
small_restaurants_df = df.drop(df[df.nr_ratings > 5000].index, inplace=False) # drop restaurants with more then 5000 reviews

# Simple linear regression of number of ratings on rating
X = small_restaurants_df['nr_ratings'].to_numpy()  # independent variable
y = small_restaurants_df['rating'].to_numpy()  # dependent variable
X = X.reshape(-1, 1)
y = y.reshape(-1, 1)
linear_regressor = LinearRegression()  #create object for the class
linear_regressor.fit(X, y)  # perform linear regression
y_pred = linear_regressor.predict(X)  # make predictions

# Create plot
plt.scatter(X, y)
plt.plot(X, y_pred, color='red')
plt.legend(['regression line'])
plt.xlabel('nr_ratings')
plt.ylabel('rating')
plt.show()

print("Intercept: \n", linear_regressor.intercept_)
print("Coefficient: ")
for coefs in linear_regressor.coef_:
    for coef in coefs:
        print(format(coef, 'f'))

The regression line indicates a negative but weak correlation of 'nr_ratings' and 'rating'. An additional review will lower a restaurants rating on average by 0.000105 stars. 

The robustness of this correlation is to doubt. There might be other factors that are correlated with the number of ratings which affect the rating (Omitted Variable Bias). What for example if the restaurant is part of a fast food chain? Then one could assume that such a place serves more daily customers and receives more ratings of which more are worse (due to lower food quality) than the average privately owned restaurant. We will reperform the regression and controll for latter in the next notebook. 

**2) Rating, Number of Ratings and Price Level**

Including price level in the plot of 'rating' and 'nr_rating' might expose more information about their coherence:

In [ ]:
%matplotlib notebook

# Scatterplot with Classification
g = sns.scatterplot(
    data=small_restaurants_df,
    x="nr_ratings", y="rating", hue="price_lvl"
    )

plt.show()

When including price level the suggested correlation seems to hold. Additionally, we see that places without a price level (i.e. "Unknown"), tend to have less reviews. One could assume that this is about places that are newly openend or not that popular.

Lets have a closer look on the distribution of 'rating' and 'nr_ratings' within each category:

In [ ]:
# Create dataframe with group means
means = df.groupby('price_lvl')[['rating', 'nr_ratings']].mean().round(decimals=2)
means = means.reindex(index=['Unknown',  # reorder ascending
                             'Inexpensive',
                             'Moderate',
                             'Expensive',
                             'Very Expensive'
                             ])

means = means.rename(columns={'rating': "mean_rating", 'nr_ratings': "mean_nr_ratings"}) # rename

# Create dataframe with group standard deviations
stds = df.groupby('price_lvl')[['rating', 'nr_ratings']].std().round(decimals=2)
stds = stds.rename(columns={'rating': "std_rating", 'nr_ratings': "std_nr_ratings"})

# Merge dataframes to output dataframe
oview = pd.concat([means, stds], axis=1)

# Add mean and standard deviation of total sample as row to output dataframe
describe_df = df.describe()
mean_rating = describe_df.loc['mean', 'rating']
mean_nr_ratings = describe_df.loc['mean', 'nr_ratings']
std_rating = describe_df.loc['std', 'rating']
std_nr_ratings = describe_df.loc['std', 'nr_ratings']

# Count frequency of each row
counter_total = describe_df.loc['count', 'rating']
counter_series = df['price_lvl'].value_counts()

# Append series as column to output dataframe
oview['freq'] = counter_series
oview.loc['Total'] = [mean_rating, mean_nr_ratings, std_rating, std_nr_ratings, counter_total]

# Format output dataframe
oview = oview.astype({'freq': int})
oview = oview.round(decimals=2)

print(oview)

<br>Worth mentioning is the above average standard deviation of ratings within the "Unknown" price category. This might be related to the fact, that not yet categorized restaurants tend to have less reviews. Since users can only give one, two, three, four or five stars the variance is on average higher when there are not many reviews. Indeed, the mean number of ratings for uncategorized is far below the sample average.

Also moticable is that restaurants within the "Moderate" and "Very Expensive" price level have been rated more often with ratings below the sample average. While for "Very Expensive" the sample size is nowhere near sufficient large, the "Moderate" priced restaurants can be acknowledged as representative. Surprisingly, most FastFood cooperations (e.g. McDonald's, Subway) are categorized as "Moderate" expensive by Google.